## REACT

In [2]:
!pip install chromadb
!pip install --upgrade chromadb
!pip install -U langchain-chroma
!pip install --upgrade openai langchain
!pip install --upgrade langchain chromadb
!pip install -U langchain-community
!pip install pysqlite3-binary
!pip install sentry-sdk==1.39.1
!pip install tqdm
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 75.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 131.4 MB/s eta 0:00:00
  Created wheel for pypika: filename=pypika-0.48.9-py2.py3-none-any.whl size=53803 sha256=7fc6c8f07cfdf37aea8c26dee86df840db35e0b8b8b4c32b16a33070bd341e06
  Stored in directory: /home/ds_user1/.cache/pip/wheels/f7/02/64/d541eac67ec459309d1fb19e727f58ecf7ffb4a8bf42d4cfe5
Successfully built pypika
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successful

In [1]:
!pwd

/project/work/flow/4.


In [7]:
import os
os.environ["CHROMA_TELEMETRY"] = "False"
import pysqlite3
import sys
sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")
import chromadb
from tqdm import tqdm

os.environ['OPENAI_API_VERSION'] = '2024-02-15-preview'
os.environ['AZURE_OPENAI_ENDPOINT'] = 'https://upappliance.openai.azure.com'
os.environ['AZURE_OPENAI_API_KEY'] = '4gWgDnxVjVaIRQhPJFn2seBhFGhsgfZhpedg6J0lmWDdsltOAlD8JQQJ99AKACYeBjFXJ3w3AAABACOGylYc'
os.environ['AZURE_OPENAI_API_DEPLOYMENT_NAME'] = "gpt-4o-mini"  # 또는 "up_gpt4o"

In [8]:
from tqdm import tqdm
import pandas as pd
import json
import os
from openai import AzureOpenAI
from new_keyword_pipeline import (
    setup_vectordb,
    format_top10_for_keyword_prompt,
    generate_new_keyword
)
from react_converted_nsc import main as sim_prompt_main

# ───────────────── 1. OpenAI 클라이언트 & 벡터 DB ──────────────────
client_openai_raw = AzureOpenAI(
    azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'],
    api_key=os.environ['AZURE_OPENAI_API_KEY'],
    api_version=os.environ['OPENAI_API_VERSION']
)
# vectordb2 = setup_vectordb("./keyword_list.csv")  # (옵션) 사용 시 주석 해제

# ───────────────── 2. 테스트 셋 로드 (필요시 행 수 조절) ───────────────
test_path = '/datasets/DTS2025000173/data/test_sample_icc_group.csv'
test_df = pd.read_csv(test_path, encoding='utf-8-sig')

# datasets/DTS2025000171/data/test_sample_new_100_group.csv
# datasets/DTS2025000172/data/test_sample_duplicate_100_group.csv
# /datasets/DTS2025000173/data/test_sample_icc_group.csv

results = []                                # 최종 결과 저장용 리스트
all_icc_list = []

# ───────────────── 3. 메인 루프 ─────────────────────────────────────
for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Processing Rows"):
    # 3-1. 지역 구분에 따라 LLM 입력 텍스트 선택
    if row["지역"] == "KR":
        input_text_for_chain = row["afterchange"]
    elif row["지역"] == "US":
        input_text_for_chain = row["beforechange"]
    else:
        print(f"[Row {idx}] Warning: 알 수 없는 '지역' 값 → afterchange 사용")
        input_text_for_chain = row["afterchange"]

    # 3-2. 파이프라인 입력 구성
    input_dict = {
        "component_group":   row["component_group"],
        "components":        row["components"],
        "original_input":    input_text_for_chain,
        "ticket_id_hashed":  row.get("ticket_id_hashed", ""),
        "지역":               row.get("지역", ""),
        "beforechange":      row.get("beforechange", ""),
        "afterchange":       row.get("afterchange", ""),
        "keyword":           row.get("keyword", "")
    }

    # 3-3. 파이프라인 실행 및 예외 처리
    try:
        (ticket_id, components, before_change, after_change,
         generated_summary, sim_score, sim_keyword,
         timing_results_run, proposal_summary_all, component_group,
         final_result, token_usage_results_run, icc_part,
         top_10_table, top_10_records) = sim_prompt_main(input_dict)

        # ICC 누적 저장
        if icc_part is not None and not icc_part.empty:
            all_icc_list.append(icc_part.copy())

        # 3-4. 키워드 후속 로직 (Top-1 유사도 % 기준 90컷)
        if sim_score >= 90:
            new_keyword = sim_keyword or row.get("keyword", "")
            print(f"[{ticket_id}] 유사도 90↑ 기존 키워드 유지: {new_keyword}")
        else:
            gen_kw_result = generate_new_keyword(
                client=client_openai_raw,
                ticket_id=ticket_id,
                component_group=component_group,
                components=components,
                beforechange=before_change,
                afterchange=after_change,
                gen_sum=generated_summary,
                sim_score=sim_score,            # 이제 Top-1 %
                sim_keyword=sim_keyword,        # 이제 Top-1 keyword
                top10_records=top_10_records,   # 필요 시 테이블 문자열도 같이 전달 가능
            )
            new_keyword = gen_kw_result["new_keyword"]
            print(f"[{ticket_id}] 유사도 90↓ 새 키워드 생성: {new_keyword}")

        # 3-5. 결과 리스트에 저장
        results.append({
            "ticket_id_hashed":  row.get('ticket_id_hashed', ''),
            "component_group":   row.get("component_group", ""),
            "components":        row.get("components", ""),
            "beforechange":      row.get("beforechange", ""),
            "afterchange":       row.get("afterchange", ""),
            "지역":              row.get("지역", ""),
            "generated_summary": generated_summary,
            "sim_score":         sim_score,              # Top-1 %
            "sim_keyword":       sim_keyword,            # Top-1 keyword
            "final_keyword":     new_keyword,
            "final_result":      final_result,           # RAG Top-2 JSON 문자열
            "timing_results":    timing_results_run,
            "token_usage_results": token_usage_results_run,
            "proposal_summary_all": proposal_summary_all,
            "top10_table_md":    top_10_table,
            # "top10_records_json": json.dumps(top_10_records, ensure_ascii=False),
        })

    except Exception as e:
        print(f"[Row {idx}] 처리 중 오류: {e}")
        continue


# ───────────────── 4. 결과 CSV 저장 ──────────────────────────────────
out_csv = "result_mini_icc_nsc_1.csv"
pd.DataFrame(results).to_csv(out_csv, index=False, encoding="utf-8-sig")
print(f"✅ 전체 완료! {out_csv} 로 저장되었습니다.")

if all_icc_list:
    icc_df_final = pd.concat(all_icc_list, ignore_index=True).drop_duplicates()
    icc_df_final.to_csv("./icc_icc_nsc_1.csv", index=False, encoding="utf-8-sig")
    print("🗂️ ICC 레코드: ./icc_icc_nsc_1.csv 저장 완료")

Processing Rows:   0%|          | 0/99 [00:00<?, ?it/s]


📥 입력 제안문:
UP가전이라고는 하는데 뭐가 UP되는지 모르겠어요.작동하는 기능은 업그레이 안되고 똑같고 멜리디나 화면만 업데이트되는거같아요.공기청정기도 마찬가지에요.
⏱️ [입력받기] 시작
✅ [입력받기] 완료 - 소요 시간: 0.00초
⏱️ [ICC 분류] 시작
✅ [ICC 분류] 완료 - 소요 시간: 0.98초
⏱️ [ICC 분기처리] 시작
👉 판별 결과 : ICC
✅ [ICC 분기처리] 완료 - 소요 시간: 0.00초
⏱️ [통합 제안 분리 및 요약 (영어)] 시작
✅ [통합 제안 분리 및 요약 (영어)] 완료 - 소요 시간: 0.98초
⏱️ [첫번째 제안 선택] 시작
✅ [첫번째 제안 선택] 완료 - 소요 시간: 1.07초
⏱️ [Top-10 유사 문서 검색] 시작
✅ [Top-10 유사 문서 검색] 완료 - 소요 시간: 4.12초
⏱️ [RAG Top-2 선택] 시작
✅ [RAG Top-2 선택] 완료 - 소요 시간: 0.00초
✅ result keys: ['component_group', 'components', 'original_input', 'ticket_id_hashed', '지역', 'beforechange', 'afterchange', 'keyword', 'icc_check', '__is_icc', 'proposal_summary_all', 'first_proposal', 'top_10_table', 'top_10_docs', 'top_10_records', 'top_10_json', 'final_result', 'ticket_id', 'sim_keyword', 'sim_score']

🔔 ICC로 판정되어 RAG Top-2 선택을 수행하지 않습니다.


Processing Rows:   1%|          | 1/99 [00:08<13:24,  8.21s/it]


✅ GPT Output:
ticket_id: 3bd5ec3
new_keyword: 건조효율

✅ [함수 내부 최종 결과]
ticket_id: 3bd5ec3
component_group: 세탁기/건조기
components: 건조기
beforechange: nan
afterchange: UP가전이라고는 하는데 뭐가 UP되는지 모르겠어요.작동하는 기능은 업그레이 안되고 똑같고 멜리디나 화면만 업데이트되는거같아요.공기청정기도 마찬가지에요.
gen_sum: 
sim_score: 0.0
sim_keyword: 
new_keyword: 건조효율
top10_used: False
[3bd5ec3] 유사도 90↓ 새 키워드 생성: 건조효율

📥 입력 제안문:
자주 청소해줬는데 팬에 낀 먼지가 보여
열어봤는데 경악을 금치못했네요
그비싼 필터를1년에 한번씩 갈아주며 가족들에게 깨끗한 공기를 마시게 해준다고 자부했는데이건 뭐 필터 할아버지가 와도 팬 청소를 안해주면 팬에 낀 먼지를 그냥 마실수밖에 없는 구조라 개선이 필요해서 이렇게 글을 남깁니다 처음구매했을때와 이벤트로 1년뒤 매니저 방문하여 청소방법설명에도 필터 교체와 외형을 물티슈로 닦는게 다였는데 솔직히 고객들이 다 열어본다며 저 같이 충격을받을듯하네요 적어도 어느정도는 셀프청소가 가능한 형태로 제작이 되어야하고 또 설치할때에도 해체방법에 대해 안내하는게 최소한의 도리라고생각이듭니다 100만 원 넘게 주고 사고 그간 매년 필터교체도 열심히해줬는데 오늘 팬 청소하며 그간 믿었던 대기업lg에 배신당한거 같아서 기분이 몹시 나쁘네요 앞으로 이런점을 꼭 개선했으면 좋겠습니다
⏱️ [입력받기] 시작
✅ [입력받기] 완료 - 소요 시간: 0.00초
⏱️ [ICC 분류] 시작
✅ [ICC 분류] 완료 - 소요 시간: 1.01초
⏱️ [ICC 분기처리] 시작
👉 판별 결과 : Proposal
✅ [ICC 분기처리] 완료 - 소요 시간: 0.00초
⏱️ [통합 제안 분리 및 요약 (영어)] 시작
✅ [통합 제안 분리 및 요약 (영어)] 완료

Processing Rows:   2%|▏         | 2/99 [00:15<12:32,  7.76s/it]

...


✅ GPT Output:
ticket_id: 80a747f
new_keyword: 세탁건조통합

✅ [함수 내부 최종 결과]
ticket_id: 80a747f
component_group: 세탁기/건조기
components: 워시타워
beforechange: nan
afterchange: 세탁물이 많을 때 인공지능 모드 세탁시 세탁통이 하우징을 치는 소리가 나는데 25kg 용량임에도 불구하고 이게 맞는 건지 회신 바랍니다
gen_sum: 
sim_score: 0.0
sim_keyword: 
new_keyword: 세탁건조통합
top10_used: False
[80a747f] 유사도 90↓ 새 키워드 생성: 세탁건조통합

📥 입력 제안문:
코스가 자동으로 없어져버려
⏱️ [입력받기] 시작
✅ [입력받기] 완료 - 소요 시간: 0.00초
⏱️ [ICC 분류] 시작
✅ [ICC 분류] 완료 - 소요 시간: 0.52초
⏱️ [ICC 분기처리] 시작
👉 판별 결과 : ICC
✅ [ICC 분기처리] 완료 - 소요 시간: 0.00초
⏱️ [통합 제안 분리 및 요약 (영어)] 시작
✅ [통합 제안 분리 및 요약 (영어)] 완료 - 소요 시간: 0.84초
⏱️ [첫번째 제안 선택] 시작
✅ [첫번째 제안 선택] 완료 - 소요 시간: 0.71초
⏱️ [Top-10 유사 문서 검색] 시작
✅ [Top-10 유사 문서 검색] 완료 - 소요 시간: 3.94초
⏱️ [RAG Top-2 선택] 시작
✅ [RAG Top-2 선택] 완료 - 소요 시간: 0.00초
✅ result keys: ['component_group', 'components', 'original_input', 'ticket_id_hashed', '지역', 'beforechange', 'afterchange', 'keyword', 'icc_check', '__is_icc', 'proposal_summary_all', 'first_proposal', 'top_10_table', 'top_10_docs', 'top_10_

Processing Rows:  99%|█████████▉| 98/99 [11:29<00:07,  7.30s/it]


✅ GPT Output:
ticket_id: 79ecf71
new_keyword: 세탁물관리

✅ [함수 내부 최종 결과]
ticket_id: 79ecf71
component_group: 세탁기/건조기
components: 세탁기
beforechange: nan
afterchange: 코스가 자동으로 없어져버려
gen_sum: 
sim_score: 0.0
sim_keyword: 
new_keyword: 세탁물관리
top10_used: False
[79ecf71] 유사도 90↓ 새 키워드 생성: 세탁물관리

📥 입력 제안문:

지금까지 사용했던 세척기에 비해 그릇수납칸이 너무 연약합니다. 그릇이 잘 지지가 안되고 심지어 쇠가 휘어지거나 칸의 크기가 변형되기도 해서 싸구려 처럼 느껴집니다.
오브제 디자인이 깔끔하고 엘지라서 믿고 샀는데 사용할수록 뒤통수맞은 느낌입니다. 


⏱️ [입력받기] 시작
✅ [입력받기] 완료 - 소요 시간: 0.00초
⏱️ [ICC 분류] 시작
✅ [ICC 분류] 완료 - 소요 시간: 0.92초
⏱️ [ICC 분기처리] 시작
👉 판별 결과 : ICC
✅ [ICC 분기처리] 완료 - 소요 시간: 0.00초
⏱️ [통합 제안 분리 및 요약 (영어)] 시작
✅ [통합 제안 분리 및 요약 (영어)] 완료 - 소요 시간: 1.98초
⏱️ [첫번째 제안 선택] 시작
✅ [첫번째 제안 선택] 완료 - 소요 시간: 0.86초
⏱️ [Top-10 유사 문서 검색] 시작
✅ [Top-10 유사 문서 검색] 완료 - 소요 시간: 2.93초
⏱️ [RAG Top-2 선택] 시작
✅ [RAG Top-2 선택] 완료 - 소요 시간: 0.00초
✅ result keys: ['component_group', 'components', 'original_input', 'ticket_id_hashed', '지역', 'beforechange', 'afterchange', 'keyword', 'icc_check', '__is_icc', 'proposal_summary_al

Processing Rows: 100%|██████████| 99/99 [11:36<00:00,  7.04s/it]


✅ GPT Output:
ticket_id: f872b6d
new_keyword: 세척효율

✅ [함수 내부 최종 결과]
ticket_id: f872b6d
component_group: 식기세척기
components: 식기세척기
beforechange: nan
afterchange: 
지금까지 사용했던 세척기에 비해 그릇수납칸이 너무 연약합니다. 그릇이 잘 지지가 안되고 심지어 쇠가 휘어지거나 칸의 크기가 변형되기도 해서 싸구려 처럼 느껴집니다.
오브제 디자인이 깔끔하고 엘지라서 믿고 샀는데 사용할수록 뒤통수맞은 느낌입니다. 


gen_sum: 
sim_score: 0.0
sim_keyword: 
new_keyword: 세척효율
top10_used: False
[f872b6d] 유사도 90↓ 새 키워드 생성: 세척효율
✅ 전체 완료! result_mini_icc_nsc_1.csv 로 저장되었습니다.
🗂️ ICC 레코드: ./icc_icc_nsc_1.csv 저장 완료


In [1]:
top_10_table

NameError: name 'top_10_table' is not defined

In [9]:
results

[]

## COT

In [ ]:
# import pandas as pd
# import os
# from openai import OpenAI
# from new_keyword_pipeline import setup_vectordb, search_top10_keywords, generate_new_keyword
# from cot import main as sim_prompt_main

# os.environ["OPENAI_API_KEY"] = "sk-"
# client = OpenAI()
# df = pd.read_csv('./all_origin_updated.csv', encoding='utf-8-sig')

# keyword_vectordb = setup_vectordb("./keyword_list.csv")

# input = {
#     "components": "로봇청소기",
#     "generated_summary": "ThinkQ 평면도상에 선을 그어 청소구역을 지정하도록 해주세요"
# }

# # sim_prompt_main에서 ICC/Proposal/유사도/키워드 추출
# result, icc_df = sim_prompt_main(df, input)

# ticket_id = result['ticket_id']
# components = result['components']
# beforechange = result['before_change']
# afterchange = result['after_change']
# gen_sum = result['generated_summary']
# ragas_score = result['ragas_score']
# sim_keyword = result['keyword']
    
# print(f"\n\n✅ [최종 결과]\n"
#       f"ticket_id: {ticket_id}\n"
#       f"components: {components}\n"
#       f"beforechange: {beforechange}\n"
#       f"afterchange: {afterchange}\n"
#       f"gen_sum: {gen_sum}\n"
#       f"RAGAS_score: {ragas_score}\n"
#       f"sim_keyword: {sim_keyword}")

# # Proposal 분기 및 처리
# if ragas_score >= 90:
#     print(f"[{ticket_id}] 유사도 90↑ 기존 키워드 업로드: {sim_keyword}")
# else:
#     top10_keywords = search_top10_keywords(keyword_vectordb, gen_sum, top_k=10)
#     result = generate_new_keyword(
#         client, ticket_id, components, beforechange, afterchange,
#         gen_sum, ragas_score, sim_keyword, top10_keywords
#     )
#     print(f"[{ticket_id}] 유사도 90↓ 새 키워드 생성: {result['new_keyword']}")

In [ ]:
flow/2_개발폴더_UP가전운영팀/phase2_hanyang/testset_inquiry_related_20250725_123340.csv